In [1]:
from moviepy.editor import ImageSequenceClip
from PIL import Image, ImageDraw, ImageFont
import os
import re

In [2]:
code_lines = ["def my_fun(a, b):", "      print(f'it is a test {a}')"]

In [3]:
# Font settings (Adjust the path to your font file)
font_path = "./fonts/ARLRDBD.ttf"  # Adjust this path to the font you want to use
font_size = 70
font = ImageFont.truetype(font_path, font_size)

In [4]:
def preprocess_line(text):
      words = []

      pattern = r'(\(|\))'
      words = re.split(pattern, text)
      result = []
      for word in words:
            if "'" not in word and "\"" not in word:
                  if " " in word:
                        pattern = r'(\s+)'
                        temp = re.split(pattern, word)
                        result += temp
                  else:
                        result.append(word)
            else:
                  if word.startswith("f"):                                              
                        result += ["f", word[1:]]
                  else:
                        result.append(word)

      return result

In [5]:
# Simulate basic syntax highlighting for Python code
def syntax_highlight(text):
    keywords = ["print", "def", "for", "in", "return", "if", "else"]
    colors = {"keyword": "blue", "string": "green", "default": "black"}
    
    highlighted_text = []
    words = preprocess_line(text)
    for word in words:
        color = colors["default"]
        if word in keywords:
            color = colors["keyword"]
        elif word.startswith("'") or word.startswith('"'):
            color = colors["string"]
        highlighted_text.append((word, color))
    return highlighted_text

In [6]:
def draw_rounded_rectangle(draw, xy, corner_radius, fill):
    """
    Draws a rounded rectangle with the specified fill color.

    :param draw: An ImageDraw instance.
    :param xy: The bounding box, as a (left, top, right, bottom)-tuple.
    :param corner_radius: Radius of the corners.
    :param fill: The fill color.
    """
    upper_left, upper_right, bottom_left, bottom_right = xy
    # Draw the center rectangle
    draw.rectangle([upper_left + corner_radius, upper_right, bottom_left - corner_radius, bottom_right], fill=fill)
    # Draw the four sides
    draw.rectangle([upper_left, upper_right + corner_radius, bottom_left, bottom_right - corner_radius], fill=fill)
    # Draw the four corners
    draw.pieslice([upper_left, upper_right, upper_left + corner_radius * 2, upper_right + corner_radius * 2], start=180, end=270, fill=fill)
    draw.pieslice([bottom_left - corner_radius * 2, upper_right, bottom_left, upper_right + corner_radius * 2], start=270, end=360, fill=fill)
    draw.pieslice([upper_left, bottom_right - corner_radius * 2, upper_left + corner_radius * 2, bottom_right], start=90, end=180, fill=fill)
    draw.pieslice([bottom_left - corner_radius * 2, bottom_right - corner_radius * 2, bottom_left, bottom_right], start=0, end=90, fill=fill)


In [7]:
def create_clip(video_name, images, fps=5, has_end_pause = False, last_frame_pause_per_second = 2):
    # Save images to a temporary directory
    temp_dir = "temp_images"
    os.makedirs(temp_dir, exist_ok=True)
    filenames = []
    for idx, img in enumerate(images):
        filename = os.path.join(temp_dir, f"{idx}.png")
        img.save(filename)
        filenames.append(filename)

    # pause a little on the last frame
    if has_end_pause:
        for _ in range(last_frame_pause_per_second*fps):
            idx = idx + 1
            filename = os.path.join(temp_dir, f"{idx}.png")
            img.save(filename)
            filenames.append(filename)
    clip = ImageSequenceClip(filenames, fps=fps)  # Adjust fps for typing speed

    # Export the video
    clip.write_videofile(video_name, codec="libx264")

    # Cleanup: remove temporary images
    for filename in filenames:
        os.remove(filename)
    os.rmdir(temp_dir)

    print(f"Video saved as {video_name}")

In [8]:
# Image settings
img_width, img_height = 1080, 1920
background_color = "white"
rect_background_color = (128, 128, 128)
rect_border_color = (0, 0, 0)
padding = 50  # Padding around the rectangle
rectangle_height = 300

In [10]:
# Generate a sequence of images
images = []
y_init = 0 + padding * 1.25
bkg_image = None
for code_line in code_lines:
    for i in range(1, len(code_line) + 1):
        if bkg_image:
            img = bkg_image.copy()
        else:
            img = Image.new("RGB", (img_width, img_height), color=background_color)
        draw = ImageDraw.Draw(img)
        
        # Draw a rectangle container
        if bkg_image is None:
            draw_rounded_rectangle(draw, [padding, padding, img_width - padding, rectangle_height], 20, fill=rect_background_color)
        
        # Apply simplified syntax highlighting
        partial_code = syntax_highlight(code_line[:i])       
        x, y = 20 + padding * 1.25, 30 + y_init  
        for word, color in partial_code:
            draw.text((x, y_init), word, fill=color, font=font)
            word_width = draw.textlength(word + " ", font=font)  # Correct method to get text size
            word_height = font_size
            x += word_width  # Move x for the next word
        images.append(img)
    y_init += font_size * 1.2
    bkg_image = img.copy()

In [11]:
create_clip("question_typing.mp4", images, fps=10, has_end_pause = True, last_frame_pause_per_second = 4)

Moviepy - Building video question_typing.mp4.
Moviepy - Writing video question_typing.mp4



Moviepy - Done !
Moviepy - video ready question_typing.mp4
Video saved as question_typing.mp4


In [12]:
# Define the choices
choices = ['Choice A: Option 1', 'Choice B: Option 2', 'Choice C: Option 3', 'Choice D: Option 4']
question_done_img = img.copy()

In [13]:
# Generate images
images = []
for num_choices in range(1, len(choices) + 1):
    img = question_done_img.copy()
    draw = ImageDraw.Draw(img)
    
    y = padding + 400 # Start y position
    for i in range(num_choices):
        draw.text((padding, y), choices[i], fill="black", font=font)
        y += font_size + 5  # Update y position for the next choice
    
    images.append(img)

In [14]:
create_clip("choices_animation.mp4", images, fps=1, has_end_pause = True, last_frame_pause_per_second = 20)

Moviepy - Building video choices_animation.mp4.
Moviepy - Writing video choices_animation.mp4



Moviepy - Done !
Moviepy - video ready choices_animation.mp4
Video saved as choices_animation.mp4


In [21]:
# Assuming 'choices' list, 'font', 'padding', and 'question_done_img' are defined
correct_choice_index = 0  # Index of the correct choice
checkmark_path = 'check-mark.png'  # Path to your checkmark image
check_mark_size = 75
# Load the checkmark image
checkmark_img = Image.open(checkmark_path).resize((check_mark_size, check_mark_size))  # Resize as needed

# Function to draw choices and optionally highlight the correct one
def draw_choices_with_highlight(img, highlight=False):
    draw = ImageDraw.Draw(img)
    y = padding + 400  # Start y position
    
    for i, choice in enumerate(choices):
        if highlight and i == correct_choice_index:
            # Calculate text size for drawing the rectangle
            text_width  = draw.textlength(choice, font=font)
            text_height = font_size
            bg_color = "#a8db81"  # A vibrant yellow, for example
            # Draw rectangle around the correct choice
            draw_rounded_rectangle(draw, [padding * 1.5 + check_mark_size , y, padding * 2.5 + check_mark_size + text_width, y + text_height*1.2], corner_radius=20, fill=bg_color)
            # Paste the checkmark image behind the answer, adjust positioning as needed
            img.paste(checkmark_img, (int(padding), y), checkmark_img)
        
        draw.text((padding * 2 + check_mark_size, y), choice, fill="black", font=font)
        y += font_size + 5  # Update y position for the next choice
    
    return img

# Generate images for animation
images = []
for num_choices in range(len(choices)):
    img = question_done_img.copy()
    images.append(draw_choices_with_highlight(img))

# Optional: Add pause frames
pause_duration_frames = 2  # Number of frames to pause before showing the answer
images.extend([images[-1]] * pause_duration_frames)

# Highlight the correct answer in the final frame(s)
img = question_done_img.copy()
highlighted_img = draw_choices_with_highlight(img, highlight=True)
images.append(highlighted_img)

In [22]:
create_clip("test_answer.mp4", images, fps=5, has_end_pause = True, last_frame_pause_per_second = 5)

Moviepy - Building video test_answer.mp4.
Moviepy - Writing video test_answer.mp4



Moviepy - Done !
Moviepy - video ready test_answer.mp4
Video saved as test_answer.mp4
